In [84]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
import base64
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aac1"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center([
        html.B(html.H1('SNHU CS-340 Dashboard'), style={'width':'1500px'}),
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
            height='200px',
            width='200px',
            ),
        html.B(html.H2('By Ryan Mitchell'), style={'width':'1500px'})
        ]),
    html.Hr(),
    html.Div(dcc.RadioItems(
            id="radio_items",
            options=[
                {'label': 'Water Rescue', 'value':'wr'},
                {'label': 'Mountain Rescue', 'value':'mr'},
                {'label': 'Disaster Rescue', 'value':'dr'},
                {'label': 'Reset', 'value':'R'}
            ], value = 'R')
            ),
    dt.DataTable(id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        style_cell={
            'whiteSpace': 'normal',
            'height': '50px',
            'minWidth': '60px', 'width':'60px', 'maxWidth':'180px',
            'textAlign': 'left'
        },
        style_data={
            'whiteSpace': 'normal',
            'height': 'auto',
            'lineHeight': '15px'
        },
        style_table={
            'overflowX': 'auto'
            
        },
        style_header={
            'backgroundColor': 'grey',
            'fontWeight': 'bold',
            'textAlign':'left'
        },
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=True,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action='native',
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
], style={'width':'2100px'})

####################
# Advanced queries #
####################
water_rescue = {
    "$and": [
        {
            "$or": [{"breed":"Labrador Retriever Mix"}, 
                    {"breed":"Chesapeake Bay Retriever"}, 
                    {"breed":"Newfoundland"}]
                    },
        {"sex_upon_outcome":"Intact Female"},
        { "$and" : [{ "age_upon_outcome_in_weeks": {"$gte": 26}}, {"age_upon_outcome_in_weeks": {"$lte": 156}}]}
    ]  
}
mountain_rescue = {
    "$and": [
        {
            "$or": [{"breed":"German Shepherd"}, 
                    {"breed":"Alaskan Malamute"},
                    {"breed":"Old English Sheepdog"},
                    {"breed":"Siberian Husky"},
                    {"breed":"Rottweiler"}]
                    },
        {"sex_upon_outcome":"Intact Male"},
        { "$and" : [{ "age_upon_outcome_in_weeks": {"$gte": 26}}, {"age_upon_outcome_in_weeks": {"$lte": 156}}]}
    ]  
}
disaster_rescue = {
    "$and": [
        {
            "$or": [{"breed":"Doberman Pinscher"}, 
                    {"breed":"German Shepherd"},
                    {"breed":"Golden Retriever"},
                    {"breed":"Bloodhound"},
                    {"breed":"Rottweiler"}]
                    },
        {"sex_upon_outcome":"Intact Male"},
        { "$and" : [{ "age_upon_outcome_in_weeks": {"$gte": 20}}, {"age_upon_outcome_in_weeks": {"$lte": 300}}]}
    ]  
}

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id','data'), 
              [Input('radio_items', 'value')])
def update_dashboard(value):
### FIX ME Add code to filter interactive data table with MongoDB queries
    if value == "R":
        df = pd.DataFrame.from_records(shelter.read({}))
        return df.to_dict('records')
    if value == "wr":
        df = pd.DataFrame.from_records(list((shelter.read(water_rescue))))
        return df.to_dict('records')
    if value == "mr":
        df = pd.DataFrame.from_records(list((shelter.read(mountain_rescue))))
        return df.to_dict('records')
    if value == "dr":
        df = pd.DataFrame.from_records(list((shelter.read(disaster_rescue))))
        return df.to_dict('records')

@app.callback(Output('datatable-id', 'style_data_conditional'), 
              [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) 
    df = pd.DataFrame.from_dict(viewData)
    
    return [
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Percentage of breeds available')
        )
    ]
    

@app.callback(Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")
    ])
def update_map(viewData):
   #FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
                dl.Tooltip(dff['breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.loc[0,'name'])
                ])
            ])
        ])
    ]



#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.



app